In [ ]:
# CSE 404 Machine Learning Project NFL combine data to HOF percentage
# Group Members: Pranav Premchand, Daphne Martin, Zeeshan Naeem, Pranesh Muthukumar

# Data in combine data folder named as qb combine data.csv

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.svm import SVR
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Step 1: Load and prepare the data
data = pd.read_csv("qb_combine_data.csv")  # Load the QB combine data

# Preprocess height column to separate feet and inches
data['Height_Feet'] = data['Ht'].apply(lambda x: int(x.split("'")[0]))
data['Height_Inches'] = data['Ht'].apply(lambda x: int(x.split("'")[1].replace('"', '')))

X = data[['Height_Feet', 'Height_Inches', 'Wt', '40yd', 'Vertical', 'Broad Jump', '3Cone', 'Shuttle']]  # Features
y = data['Hof']  # Target variable (continuous)

# Normalize the features
X = (X - X.mean()) / X.std()

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2.1: Handle missing values with mean imputation
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
# Step 3: Build and train the support vector regression model
svr_model = SVR()  # Default SVR
svr_model.fit(X_train_imputed, y_train)

# Step 4: Build and train the Ridge (L2 regularization) model
ridge_model = Ridge(alpha=0.1)  # You can adjust the alpha parameter for tuning the strength of regularization
ridge_model.fit(X_train_imputed, y_train)

# Step 5: Build and train the Lasso (L1 regularization) model
lasso_model = Lasso(alpha=0.1)  # You can adjust the alpha parameter for tuning the strength of regularization
lasso_model.fit(X_train_imputed, y_train)

# Step 6: Build the neural network regression model with increased complexity and more advanced architecture
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_imputed.shape[1],)),  # Input layer
    Dropout(0.2),  # Dropout layer to prevent overfitting
    Dense(64, activation='relu'),  # Hidden layer
    Dropout(0.2),  # Dropout layer to prevent overfitting
    Dense(32, activation='relu'),  # Hidden layer
    Dropout(0.2),  # Dropout layer to prevent overfitting
    Dense(16, activation='relu'),  # Hidden layer
    Dropout(0.2),  # Dropout layer to prevent overfitting
    Dense(1)  # Output layer
])

# Step 7: Compile the neural network model
nn_model.compile(optimizer='adam', loss='mean_squared_error')

# Step 8: Train the neural network model
nn_model.fit(X_train_imputed, y_train, epochs=20, batch_size=64, validation_split=0.2)

# Step 9: Evaluate the models
svr_mse = mean_squared_error(y_test, svr_model.predict(X_test_imputed))
ridge_mse = mean_squared_error(y_test, ridge_model.predict(X_test_imputed))
lasso_mse = mean_squared_error(y_test, lasso_model.predict(X_test_imputed))
nn_mse = mean_squared_error(y_test, nn_model.predict(X_test_imputed))

print("Support Vector Regression Mean Squared Error:", svr_mse)
print("Ridge Regression Mean Squared Error:", ridge_mse)
print("Lasso Regression Mean Squared Error:", lasso_mse)
print("Neural Network Mean Squared Error:", nn_mse)

# Step 10: Define function to predict HOF percentage for a single player's combine data
def predict_hof_percentage(height_feet, height_inches, weight, forty_yard, vertical_jump, broad_jump, three_cone, shuttle):
    new_data = pd.DataFrame([[height_feet, height_inches, weight, forty_yard, vertical_jump, broad_jump, three_cone, shuttle]],
                            columns=['Height_Feet', 'Height_Inches', 'Wt', '40yd', 'Vertical', 'Broad Jump', '3Cone', 'Shuttle'])
    new_data_imputed = imputer.transform(new_data)
    return svr_model.predict(new_data_imputed)[0]

# Step 11: Example usage of the prediction function
height_feet = 6
height_inches = 4
weight = 211  # pounds
forty_yard = 5.28  # seconds
vertical_jump = 24.5  # inches
broad_jump = 99  # inches
three_cone = 7.2  # seconds
shuttle = 4.38  # seconds

hof_percentage = predict_hof_percentage(height_feet, height_inches, weight, forty_yard, vertical_jump, broad_jump, three_cone, shuttle)
print("Predicted Hall of Fame Percentage:", (hof_percentage)*100)


Epoch 1/20
3/3 [==============================] - 4s 287ms/step - loss: 0.0242 - val_loss: 0.0723
Epoch 2/20
3/3 [==============================] - 0s 63ms/step - loss: 0.0256 - val_loss: 0.0707
Epoch 3/20
3/3 [==============================] - 0s 53ms/step - loss: 0.0192 - val_loss: 0.0700
Epoch 4/20
3/3 [==============================] - 0s 100ms/step - loss: 0.0208 - val_loss: 0.0699
Epoch 5/20
3/3 [==============================] - 0s 43ms/step - loss: 0.0196 - val_loss: 0.0701
Epoch 6/20
3/3 [==============================] - 0s 67ms/step - loss: 0.0177 - val_loss: 0.0702
Epoch 7/20
3/3 [==============================] - 0s 54ms/step - loss: 0.0168 - val_loss: 0.0704
Epoch 8/20
3/3 [==============================] - 0s 44ms/step - loss: 0.0186 - val_loss: 0.0701
Epoch 9/20
3/3 [==============================] - 0s 49ms/step - loss: 0.0174 - val_loss: 0.0698
Epoch 10/20
3/3 [==============================] - 0s 31ms/step - loss: 0.0175 - val_loss: 0.0695
Epoch 11/20
3/3 [==========